In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from package.utils import DotDict, timer, get_config, load_data, merge_data

# from package.cluster_analysis import plot_elbow, plot_box_cluster, plot_box_cluster_sns, show_size_cluster, validate_decisiontree_cluster, plot_centroidsim_cluster, cal_featureimportance, plot_pca

from package.clustering import CONFIG_PATH

conf = get_config(CONFIG_PATH)

function: get_config is starting...
function: get_config successfully executed at 0.0014829635620117188s


In [3]:
path = 's3://tli-crm-segmentation/data-zones/neutral/grouped/feature_99pct.csv'
raw_df = load_data(path=path).set_index(conf.PARTY_ID_COLUMN)

path = 's3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_99pct.csv'
scaled_df = load_data(path=path).set_index(conf.PARTY_ID_COLUMN)

function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_99pct.csv


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:283: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_99pct.csv
function: load_data successfully executed at 1.3066892623901367s
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_99pct.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_99pct.csv
function: load_data successfully executed at 0.40511369705200195s


In [4]:
landing_path = 's3://tli-crm-segmentation/data-test/load_dt=20250424'

In [5]:
X = scaled_df.copy()

In [6]:
from package.clustering.get_model import get_model
from package.clustering.get_log import Log
from package.clustering.predict import predict_model, add_labels

@timer
def train_model(model, X):
    return model.fit(X)

In [7]:
import pickle
import s3fs

label_df = None

for k in range(2, 11):
    model_name = 'kmeans'
    model_param = {'n_clusters': k, 'random_state': 0}

    model = get_model(method=model_name, params=model_param)
    model = train_model(model=model, X=X)
    labels = predict_model(model=model, method=model_name, X=X)
    result = add_labels(X=X, labels=labels)
    result = result.rename(columns={'labels':f'K{k}'})
    result = result.set_index(conf.PARTY_ID_COLUMN)
    label_df = pd.concat(objs=[label_df, result], axis=1)
    
    s3_path = f'{landing_path}/k{k}.pkl'
    fs = s3fs.S3FileSystem()
    with fs.open(s3_path, 'wb') as f:
        pickle.dump(model, f)

function: get_model is starting...
function: get_model successfully executed at 9.894371032714844e-05s
function: train_model is starting...
function: train_model successfully executed at 3.1820738315582275s
function: predict_model is starting...
function: predict_model successfully executed at 0.0010433197021484375s
function: get_model is starting...
function: get_model successfully executed at 5.364418029785156e-05s
function: train_model is starting...
function: train_model successfully executed at 0.1380167007446289s
function: predict_model is starting...
function: predict_model successfully executed at 0.0010266304016113281s
function: get_model is starting...
function: get_model successfully executed at 4.553794860839844e-05s
function: train_model is starting...
function: train_model successfully executed at 0.1347339153289795s
function: predict_model is starting...
function: predict_model successfully executed at 0.0009455680847167969s
function: get_model is starting...
function: g

In [8]:
# x_scaled_df = pd.DataFrame(scaled_df, columns=[f'SCALED_{col}' for col in scaled_df.columns],index=scaled_df.index)

In [9]:
# x_scaled_df.columns

In [10]:
raw_df.columns = [col.upper() for col in raw_df.columns]
scaled_df.columns = [f'SCALED_{col}'.upper() for col in scaled_df.columns]

In [11]:
main_df = pd.concat(objs=[raw_df, scaled_df, label_df], axis=1)
main_df.index.name = 'PARTY_RK'
main_df = main_df.reset_index()
main_df.head()

,PARTY_RK,POLICY_COUNT_SAVING,TOTAL_PREMIUM_SAVING,POLICY_COUNT_LEGACY,TOTAL_PREMIUM_LEGACY,POLICY_COUNT_OTHER,TOTAL_PREMIUM_OTHER,IS_BENE_PARENT,IS_BENE_CHILD,IS_BENE_SPOUSE,...,SCALED_TOTAL_PREMIUM,K2,K3,K4,K5,K6,K7,K8,K9,K10
0,8000002493,4.0,436089.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,-0.007327,0,1,1,3,3,3,3,3,9
1,8000004231,1.0,985000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.621245,0,1,1,1,1,1,1,1,1
2,8000004232,1.0,985000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.621245,0,1,1,1,1,1,1,1,1
3,8000005828,3.0,172450.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-0.274157,1,0,0,0,0,0,0,0,0
4,8000006190,3.0,219620.0,1.0,359400.0,0.0,0.0,0.4,0.2,0.4,...,0.133930,0,1,1,1,1,1,1,1,1


In [12]:
file_name = 'EXPERIMENT_CRM_SEGMENTATION_99PCT_KMEANS'
main_df.to_csv(f'{landing_path}/{file_name}.csv', index=False)